In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2


In [89]:
import itertools

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
import warnings

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit
from prophet import Prophet
from prophet.diagnostics import cross_validation, performance_metrics

from src.features import build_features
from src.models import train_model

warnings.filterwarnings('ignore')
sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

In [90]:
DIAGS_RELEVANTES = [
    "C33X",
    "C340",
    "C341",
    "C342",
    "C343",
    "C381",
    "C384",
    "C450",
    "C780",
    "C782",
    "D143",
    "D381",
    "E848",
    "I080",
    "I081",
    "I340",
    "I351",
    "I352",
    "I420",
    "I456",
    "I472",
    "I495",
    "I710",
    "I712",
    "J47X",
    "J679",
    "J841",
    "J848",
    "J849",
    "J860",
    "J869",
    "J955",
    "M348",
    "T820",
    "T821",
    "Z450",
]

In [91]:
egresos_torax = pd.read_csv("../data/processed/egresos_torax_mes_y_dia.csv")
egresos_torax["FECHA_EGRESO"] = pd.to_datetime(
    egresos_torax["ANO_EGRESO"].astype(str)
    + "-"
    + egresos_torax["MES_EGRESO"].astype(str)
    + "-"
    + egresos_torax["DIA_EGRESO"].astype(str),
    format="%Y-%m-%d",
)

# 1. Proyección de Egresos Hospitalarios para Diagnósticos más frecuentes en el INT

En este cuadernillo se quiere crear un modelo para estimar la proyección de egresos
hospitalarios para los diagnósticos más relevantes para el INT hasta el año 2035. Para realizar
la proyección se utilizará el modelo Prophet, y se hará una proyección por día.

Para este análisis se obviará el año 2019 para entrenar/testear el modelo. Esto, ya que es un
año anómalo.

In [92]:
df = egresos_torax.query("ANO_EGRESO <= 2019")
df = df[df["DIAG1"].isin(DIAGS_RELEVANTES)]
df = df.drop(
    columns={
        "ANO_EGRESO",
        "MES_EGRESO",
        "DIA_EGRESO",
        "ESTABLECIMIENTO_SALUD",
        "GLOSA_ESTABLECIMIENTO_SALUD",
    }
)


# Agrupa por día y genera un calendario completo hasta el 2019 (imputa días faltantes)
df = df.groupby("FECHA_EGRESO").sum().resample("D").sum()
df = df.drop(columns="DIAG1")

## 1.1 Preprocesamiento para Modelo Prophet

El modelo prophet requiere 2 columnas para hacer la proyección

In [93]:
df_prophet = (
    df.copy()
    .reset_index()[["FECHA_EGRESO", "n_egresos"]]
    .rename(columns={"FECHA_EGRESO": "ds", "n_egresos": "y"})
)

## 1.2 Separación de muestra y Modelamiento

In [94]:
CORTE_VALID = "2019-01-01"

train = df_prophet.query("ds < @CORTE_VALID")
test = df_prophet.query("ds >= @CORTE_VALID")

In [104]:
param_grid = {
    "changepoint_prior_scale": [0.001, 0.01, 0.1, 0.5],
    "seasonality_prior_scale": [0.01, 0.1, 1.0, 10.0],
    "holidays_prior_scale": [0.01, 0.1, 1.0, 10.0],
    # "seasonality_mode": ["additive", "multiplicative"],
}

In [123]:
# Genera todas las combinaciones de hiperparametros
all_params = [dict(zip(param_grid.keys(), v)) for v in itertools.product(*param_grid.values())]
rmses = []
rmses_val = []

In [124]:
CUTOFFS = pd.to_datetime(["2014-01-01", "2015-01-01", "2016-01-01", "2017-01-01"])

# Entrena con validacion cruzada
for params in all_params:
    modelo = Prophet(**params).add_country_holidays("Chile").fit(train)
    df_cv = cross_validation(modelo, cutoffs=CUTOFFS, horizon="365 days", parallel="processes")
    df_p = performance_metrics(df_cv, rolling_window=1)
    rmses.append(df_p['rmse'].values[0])
    
    yhat_val = modelo.predict(test.drop(columns="y"))["yhat"]
    rmse_val = np.sqrt(mean_squared_error(test["y"], yhat_val))
    rmses_val.append(rmse_val)
    
# Encuentra los mejores parametros
tuning_results = pd.DataFrame(all_params)
tuning_results["rmse"] = rmses
tuning_results["rmse_val"] = rmses_val

14:01:19 - cmdstanpy - INFO - Chain [1] start processing
14:01:19 - cmdstanpy - INFO - Chain [1] done processing
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interactive plots will not work.
14:01:20 - cmdstanpy - INFO - Chain [1] start processing
14:01:20 - cmdstanpy - INFO - Chain [1] start processing
14:01:20 - cmdstanpy - INFO - Chain [1] start processing
14:01:20 - cmdstanpy - INFO - Chain [1] start processing
14:01:20 - cmdstanpy - INFO - Chain [1] done processing
14:01:20 - cmdstanpy - INFO - Chain [1] done processing
14:01:20 - cmdstanpy - INFO - Chain [1] done processing
14:01:20 - cmdstanpy - INFO - Chain [1] done processing
14:01:21 - cmdstanpy - INFO - Chain [1] start processing
14:01:21 - cmdstanpy - INFO - Chain [1] done processing
Importing plotly failed. Interactive plots will not work.
Importing plotly failed. Interac

In [129]:
best_params = all_params[np.argmin(rmses_val)]
print(best_params)

{'changepoint_prior_scale': 0.1, 'seasonality_prior_scale': 1.0, 'holidays_prior_scale': 1.0}


In [130]:
modelo_optimo = Prophet(**best_params).add_country_holidays("Chile").fit(train)

14:06:10 - cmdstanpy - INFO - Chain [1] start processing
14:06:10 - cmdstanpy - INFO - Chain [1] done processing


In [131]:
yhat = modelo_optimo.predict(test.drop(columns="y"))
rmse = np.sqrt(mean_squared_error(test["y"], yhat["yhat"]))
print(f"El RMSE con el conjunto de validación fue de: {rmse:.2f}")

El RMSE con el conjunto de validación fue de: 2.66
